In [1]:
import datetime
import os
import pickle
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
from sklearn.ensemble import AdaBoostClassifier
from sklearn.inspection import PartialDependenceDisplay, partial_dependence
from sklearn.metrics import (ConfusionMatrixDisplay, accuracy_score, precision_recall_curve, roc_auc_score, roc_curve)
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.tree import DecisionTreeClassifier

import shap

%config InlineBackend.figure_format = 'retina'

In [2]:
from notebook import notebookapp
import urllib
import json
import ipykernel
from shutil import copy2

def notebook_path():
    """Returns the absolute path of the Notebook or None if it cannot be determined
    NOTE: works only when the security is token-based or there is also no password
    """
    connection_file = os.path.basename(ipykernel.get_connection_file())
    kernel_id = connection_file.split('-', 1)[1].split('.')[0]

    for srv in notebookapp.list_running_servers():
        try:
            if srv['token']=='' and not srv['password']:  # No token and no password, ahem...
                req = urllib.request.urlopen(srv['url']+'api/sessions')
            else:
                req = urllib.request.urlopen(srv['url']+'api/sessions?token='+srv['token'])
            sessions = json.load(req)
            for sess in sessions:
                if sess['kernel']['id'] == kernel_id:
                    return os.path.join(srv['notebook_dir'],sess['notebook']['path'])
        except:
            pass  # There may be stale entries in the runtime directory 
    return None


def copy_current_nb(new_name):
    nb = notebook_path()
    if nb:
        new_path = os.path.join(os.path.dirname(nb), new_name+'.ipynb')
        copy2(nb, new_path)
    else:
        print("Current notebook path cannot be determined.")

In [3]:
df = pd.read_csv('Data/cover_type_engineered.csv')

In [4]:
df = df.loc[:, [col for col in df if not col.startswith('Cover_Type_')]]
X = df.drop(columns=['Cover_Type', 'Aspect_Sector'])
y = df['Cover_Type'] - 1

In [5]:
X = X.loc[:, [col for col in X if not col.startswith('Soil_Type')]]

In [ ]:
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M")

warnings.filterwarnings("ignore", category=FutureWarning, module="pandas.api.types")

# Assuming X and y are defined
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

# Define the base estimator for AdaBoost with a more complex decision tree
base_estimator = DecisionTreeClassifier()  

# Define the AdaBoost classifier using the base estimator
estimator = AdaBoostClassifier(estimator=base_estimator, algorithm='SAMME')

# Define hyperparameters for tuning both AdaBoost and its base estimator
hyperparameters = {
    "estimator__criterion": ["gini", "entropy"],
    "estimator__splitter": ["best", "random"],
    "estimator__max_features": [None, "sqrt", "log2"],
    "estimator__max_depth": [7, 8, 9, 10, 11, 12],  
    "n_estimators": stats.randint(50, 150),
    "learning_rate": stats.uniform(0.01, 0.5),
}


random_search = RandomizedSearchCV(estimator,
                                   param_distributions=hyperparameters,
                                   scoring='accuracy',
                                   return_train_score=True,
                                   n_iter=500,
                                   cv=5,
                                   verbose=10,
                                   n_jobs=-1)

# Fit the RandomizedSearchCV
try:
    random_search.fit(X_train, y_train)  # Assuming X_train and y_train are defined
    print("Best parameters found:", random_search.best_params_)
    print("Best score found:", random_search.best_score_)

    
    # Save results
    results_path = f"./tuning_results/tuning_ada/{timestamp}"
    if not os.path.exists(results_path):
        os.makedirs(results_path)
        
    # Saving cross-validation results
    cv_results = pd.DataFrame(random_search.cv_results_)
    cv_results_file = f"{timestamp}_results.csv"
    cv_results.to_csv(os.path.join(results_path, cv_results_file), index=False)
    
    # Save .ipynb
    copy_current_nb(os.path.join(results_path, 'Evaluation_Notebook'))
    
    # Save Model
    file_name = f"ada_{timestamp}.pkl"
    pickle.dump(random_search, open(os.path.join(results_path, file_name), "wb"))
    
    # random_search = pickle.load(open(file_name, "rb"))
    

except Exception as e:
    print(f"An error occurred during model optimization: {e}")


Fitting 5 folds for each of 500 candidates, totalling 2500 fits
[CV 2/5; 2/500] START estimator__criterion=entropy, estimator__max_depth=8, estimator__max_features=log2, estimator__splitter=random, learning_rate=0.35826977122023296, n_estimators=55
[CV 2/5; 2/500] END estimator__criterion=entropy, estimator__max_depth=8, estimator__max_features=log2, estimator__splitter=random, learning_rate=0.35826977122023296, n_estimators=55;, score=(train=0.865, test=0.766) total time=   0.6s
[CV 1/5; 3/500] START estimator__criterion=gini, estimator__max_depth=10, estimator__max_features=log2, estimator__splitter=random, learning_rate=0.1341728939072202, n_estimators=124
[CV 1/5; 3/500] END estimator__criterion=gini, estimator__max_depth=10, estimator__max_features=log2, estimator__splitter=random, learning_rate=0.1341728939072202, n_estimators=124;, score=(train=0.986, test=0.803) total time=   1.4s
[CV 4/5; 3/500] START estimator__criterion=gini, estimator__max_depth=10, estimator__max_features=

[CV 3/5; 2/500] START estimator__criterion=entropy, estimator__max_depth=8, estimator__max_features=log2, estimator__splitter=random, learning_rate=0.35826977122023296, n_estimators=55
[CV 3/5; 2/500] END estimator__criterion=entropy, estimator__max_depth=8, estimator__max_features=log2, estimator__splitter=random, learning_rate=0.35826977122023296, n_estimators=55;, score=(train=0.850, test=0.756) total time=   0.6s
[CV 5/5; 2/500] START estimator__criterion=entropy, estimator__max_depth=8, estimator__max_features=log2, estimator__splitter=random, learning_rate=0.35826977122023296, n_estimators=55
[CV 5/5; 2/500] END estimator__criterion=entropy, estimator__max_depth=8, estimator__max_features=log2, estimator__splitter=random, learning_rate=0.35826977122023296, n_estimators=55;, score=(train=0.861, test=0.743) total time=   0.6s
[CV 3/5; 3/500] START estimator__criterion=gini, estimator__max_depth=10, estimator__max_features=log2, estimator__splitter=random, learning_rate=0.1341728939

[CV 4/5; 1/500] START estimator__criterion=entropy, estimator__max_depth=9, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.3974463633813646, n_estimators=136
[CV 4/5; 1/500] END estimator__criterion=entropy, estimator__max_depth=9, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.3974463633813646, n_estimators=136;, score=(train=1.000, test=0.831) total time=   6.5s
[CV 4/5; 5/500] START estimator__criterion=entropy, estimator__max_depth=8, estimator__max_features=None, estimator__splitter=random, learning_rate=0.14768964738808044, n_estimators=53
[CV 4/5; 5/500] END estimator__criterion=entropy, estimator__max_depth=8, estimator__max_features=None, estimator__splitter=random, learning_rate=0.14768964738808044, n_estimators=53;, score=(train=0.934, test=0.841) total time=   1.8s
[CV 5/5; 6/500] START estimator__criterion=gini, estimator__max_depth=12, estimator__max_features=None, estimator__splitter=best, learning_rate=0.1444227417443651

[CV 3/5; 1/500] START estimator__criterion=entropy, estimator__max_depth=9, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.3974463633813646, n_estimators=136
[CV 3/5; 1/500] END estimator__criterion=entropy, estimator__max_depth=9, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.3974463633813646, n_estimators=136;, score=(train=0.999, test=0.848) total time=   5.9s
[CV 4/5; 4/500] START estimator__criterion=gini, estimator__max_depth=7, estimator__max_features=sqrt, estimator__splitter=best, learning_rate=0.1352570191282893, n_estimators=93
[CV 4/5; 4/500] END estimator__criterion=gini, estimator__max_depth=7, estimator__max_features=sqrt, estimator__splitter=best, learning_rate=0.1352570191282893, n_estimators=93;, score=(train=0.943, test=0.828) total time=   3.4s
[CV 1/5; 7/500] START estimator__criterion=gini, estimator__max_depth=8, estimator__max_features=None, estimator__splitter=random, learning_rate=0.07465490319828542, n_estima

[CV 1/5; 41/500] END estimator__criterion=entropy, estimator__max_depth=9, estimator__max_features=None, estimator__splitter=best, learning_rate=0.4871543283584074, n_estimators=114;, score=(train=1.000, test=0.867) total time=  46.5s
[CV 2/5; 45/500] START estimator__criterion=entropy, estimator__max_depth=7, estimator__max_features=sqrt, estimator__splitter=best, learning_rate=0.4243656283530011, n_estimators=109
[CV 2/5; 45/500] END estimator__criterion=entropy, estimator__max_depth=7, estimator__max_features=sqrt, estimator__splitter=best, learning_rate=0.4243656283530011, n_estimators=109;, score=(train=0.956, test=0.821) total time=   6.6s
[CV 5/5; 46/500] START estimator__criterion=gini, estimator__max_depth=11, estimator__max_features=None, estimator__splitter=random, learning_rate=0.26009375445888555, n_estimators=65
[CV 5/5; 46/500] END estimator__criterion=gini, estimator__max_depth=11, estimator__max_features=None, estimator__splitter=random, learning_rate=0.260093754458885

...

[CV 5/5; 369/500] END estimator__criterion=gini, estimator__max_depth=12, estimator__max_features=None, estimator__splitter=best, learning_rate=0.467579555334045, n_estimators=63;, score=(train=1.000, test=0.871) total time=  20.8s
[CV 2/5; 371/500] START estimator__criterion=entropy, estimator__max_depth=11, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.33759930466678784, n_estimators=103
[CV 2/5; 371/500] END estimator__criterion=entropy, estimator__max_depth=11, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.33759930466678784, n_estimators=103;, score=(train=1.000, test=0.841) total time=   6.3s
[CV 4/5; 371/500] START estimator__criterion=entropy, estimator__max_depth=11, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.33759930466678784, n_estimators=103
[CV 4/5; 371/500] END estimator__criterion=entropy, estimator__max_depth=11, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.3375993

[CV 3/5; 332/500] END estimator__criterion=entropy, estimator__max_depth=12, estimator__max_features=None, estimator__splitter=random, learning_rate=0.5040154534438669, n_estimators=134;, score=(train=1.000, test=0.853) total time=  10.0s
[CV 3/5; 336/500] START estimator__criterion=entropy, estimator__max_depth=10, estimator__max_features=log2, estimator__splitter=random, learning_rate=0.4074698996459015, n_estimators=59
[CV 3/5; 336/500] END estimator__criterion=entropy, estimator__max_depth=10, estimator__max_features=log2, estimator__splitter=random, learning_rate=0.4074698996459015, n_estimators=59;, score=(train=0.976, test=0.775) total time=   1.1s
[CV 4/5; 337/500] START estimator__criterion=entropy, estimator__max_depth=12, estimator__max_features=None, estimator__splitter=random, learning_rate=0.4943028631494309, n_estimators=85
[CV 4/5; 337/500] END estimator__criterion=entropy, estimator__max_depth=12, estimator__max_features=None, estimator__splitter=random, learning_rate=

[CV 4/5; 344/500] END estimator__criterion=gini, estimator__max_depth=9, estimator__max_features=sqrt, estimator__splitter=random, learning_rate=0.3796108500197106, n_estimators=81;, score=(train=0.973, test=0.795) total time=   1.1s
[CV 1/5; 345/500] START estimator__criterion=entropy, estimator__max_depth=7, estimator__max_features=None, estimator__splitter=random, learning_rate=0.3910935082067484, n_estimators=144
[CV 1/5; 345/500] END estimator__criterion=entropy, estimator__max_depth=7, estimator__max_features=None, estimator__splitter=random, learning_rate=0.3910935082067484, n_estimators=144;, score=(train=0.949, test=0.815) total time=   4.5s
[CV 2/5; 346/500] START estimator__criterion=gini, estimator__max_depth=11, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.1409346396356166, n_estimators=139
[CV 2/5; 346/500] END estimator__criterion=gini, estimator__max_depth=11, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.140934639635

[CV 1/5; 364/500] END estimator__criterion=entropy, estimator__max_depth=7, estimator__max_features=None, estimator__splitter=random, learning_rate=0.38439180114515326, n_estimators=65;, score=(train=0.923, test=0.814) total time=   2.1s
[CV 2/5; 365/500] START estimator__criterion=gini, estimator__max_depth=11, estimator__max_features=None, estimator__splitter=best, learning_rate=0.30032504883239974, n_estimators=89
[CV 2/5; 365/500] END estimator__criterion=gini, estimator__max_depth=11, estimator__max_features=None, estimator__splitter=best, learning_rate=0.30032504883239974, n_estimators=89;, score=(train=1.000, test=0.890) total time=  27.5s
[CV 1/5; 367/500] START estimator__criterion=gini, estimator__max_depth=8, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.19597358060045234, n_estimators=71
[CV 1/5; 367/500] END estimator__criterion=gini, estimator__max_depth=8, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.19597358060045234,

[CV 2/5; 372/500] END estimator__criterion=gini, estimator__max_depth=8, estimator__max_features=None, estimator__splitter=random, learning_rate=0.4292239922149854, n_estimators=96;, score=(train=0.988, test=0.850) total time=   3.4s
[CV 5/5; 372/500] START estimator__criterion=gini, estimator__max_depth=8, estimator__max_features=None, estimator__splitter=random, learning_rate=0.4292239922149854, n_estimators=96
[CV 5/5; 372/500] END estimator__criterion=gini, estimator__max_depth=8, estimator__max_features=None, estimator__splitter=random, learning_rate=0.4292239922149854, n_estimators=96;, score=(train=0.985, test=0.834) total time=   3.3s
[CV 4/5; 373/500] START estimator__criterion=gini, estimator__max_depth=11, estimator__max_features=sqrt, estimator__splitter=best, learning_rate=0.04376331040581948, n_estimators=138
[CV 4/5; 373/500] END estimator__criterion=gini, estimator__max_depth=11, estimator__max_features=sqrt, estimator__splitter=best, learning_rate=0.04376331040581948, 

In [ ]:
cv_results.head()

In [ ]:
results_path = f"./tuning_results/tuning_ada/{timestamp}/Assets"
if not os.path.exists(results_path):
    os.makedirs(results_path)

# CV Evaluation

In [ ]:
cv_results.columns

In [ ]:
cv_results.sort_values(by='rank_test_score', ascending=True).head(5)

In [ ]:
sorted_cv = cv_results.sort_values(by='rank_test_score', ascending=True)

# Train vs Test Comparison

In [ ]:
plt.figure(figsize=(16, 6))    

plt.plot(sorted_cv['rank_test_score'], sorted_cv['mean_train_score'], label="Train Score")
plt.plot(sorted_cv['rank_test_score'], sorted_cv['mean_test_score'], label="Validation Score")

plt.grid()
plt.xlabel('Sorted Validation Rank')
plt.ylabel('Accuracy')
plt.title('Train and Test Accuracy by Final Rank')
plt.legend(loc='best')

filename = "test_train_by_rank.png"
plt.savefig(os.path.join(results_path, filename))

plt.show()

In [ ]:
# boxplot algorithm comparison
fig = plt.figure(figsize=(10, 3))
fig.suptitle('Test Accuracy by Rank')

ax = fig.add_subplot(111)

plt.boxplot(sorted_cv.iloc[:10, :][['split0_test_score', 'split1_test_score', 'split2_test_score',
   'split3_test_score', 'split4_test_score']].T)
ax.set_xticklabels(range(1, 11))
ax.set_xlabel("Rank")
ax.set_ylabel("Accuracy")

filename = "test_accuracy_by_rank.png"
plt.savefig(os.path.join(results_path, filename))

plt.show()

In [ ]:
# boxplot algorithm comparison
fig = plt.figure(figsize=(10, 3))
fig.suptitle('Train Accuracy by Rank')

ax = fig.add_subplot(111)

plt.boxplot(sorted_cv.iloc[:10, :][['split0_train_score', 'split1_train_score', 'split2_train_score',
   'split3_train_score', 'split4_train_score']].T)
ax.set_xticklabels(range(1, 11))
ax.set_xlabel("Rank")
ax.set_ylabel("Accuracy")

filename = "train_accuracy_by_rank.png"
plt.savefig(os.path.join(results_path, filename))

plt.show()

In [ ]:
max_params = cv_results.loc[cv_results['rank_test_score'] == 1]
best_params = max_params.params.values[0]

In [ ]:
print(f"Mean Train set, Accuracy = {max_params['mean_train_score'].values[0]:.2f}")
print(f"Mean Test  set, Accuracy = {max_params['mean_test_score'].values[0]:.2f}")

In [ ]:
random_search = pickle.load(open(os.path.join(f'./tuning_results/tuning_ada/{timestamp}/', file_name), "rb"))
model = random_search.best_estimator_

#model = ADAClassifier(**best_params)
#model.fit(X_train, y_train)

y_train_prediction = model.predict(X_train)
y_test_prediction = model.predict(X_test)

In [ ]:
print(f"Train set, Accuracy = {accuracy_score(y_train, y_train_prediction):.2f}")
print(f"Test set, Accuracy = {accuracy_score(y_test, y_test_prediction):.2f}")

In [ ]:
ind = np.argpartition(model.feature_importances_, -20)[-20:]

features = X.columns[ind]
importance = model.feature_importances_[ind]

fig = plt.figure(figsize=(12, 6))
plt.barh(range(len(ind)), importance, align='center')
plt.yticks(range(len(ind)), features)
plt.title('Feature Importance ADA')
plt.grid()

filename = "feature_importance.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()

In [ ]:
# TEST
for col in ['param_estimator__max_depth', 'param_n_estimators', 'param_learning_rate']:
    
    plt.figure(figsize=(16, 6))    

    m, b = np.polyfit(list(sorted_cv['mean_test_score'].values), list(sorted_cv[col].values), 1)
    plt.plot(sorted_cv['mean_test_score'], m * sorted_cv['mean_test_score'] + b, c='r', label="Regression Line")
    plt.scatter(sorted_cv['mean_test_score'], sorted_cv[col], label=f"{col} Values")
    
    plt.grid()
    plt.xlabel('Mean Validation Score')
    plt.ylabel('Parameter Value')
    plt.title(col)
    plt.legend(loc='best')

    
    filename = f"{col}_by_rank.png"
    plt.savefig(os.path.join(results_path, filename))
                  
    plt.show()


# Hyperparameter Evaluation

In [ ]:

def plot_parameters(x_values, title):
    
    fig, ax1 = plt.subplots(figsize=(16, 6))
    ax2 = ax1.twinx()

    ax1.scatter(x_values, cv_results['mean_test_score'], label='mean_test_score', c='b')
    #ax2.scatter(x_values, cv_results['std_test_score'], label='std_test_score', c='r')

    m, b = np.polyfit(list(x_values.values), list(cv_results['mean_test_score'].values), 1)
    ax1.plot(x_values, m * x_values + b, c='b')

    m, b = np.polyfit(list(x_values.values), list(cv_results['std_test_score'].values), 1)
    ax2.plot(x_values, m * x_values + b, c='r', label='std_test_score')
    
    ax1.set_title(title)
    ax1.set_xlabel('Parameter Value')
    ax1.set_ylabel('Mean Test Score')
    ax2.set_ylabel('Standard Deviation of Test Score')
    ax1.grid(True)
    
    
    # Combine the legends from both axes
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='upper right')

    filename = f"{title}_test_score.png"
    plt.savefig(os.path.join(results_path, filename))
            
    plt.show()



In [ ]:

for param in ['param_estimator__max_depth', 'param_n_estimators', 'param_learning_rate']:

    x_values = cv_results[param]
    plot_parameters(x_values, param)

# Plotting Evaluation Metrics (Precision, Recall, F1-Score, AUC-ROC):


In [ ]:

# For multiclass classification, you need to binarize the labels
y_true_bin = label_binarize(y_test, classes=np.unique(y_test))
y_score_bin = label_binarize(y_test_prediction, classes=np.unique(y_test_prediction))

auc_roc = roc_auc_score(y_true_bin, y_score_bin, average='macro')

# Plot Precision-Recall curve for each class
precision = dict()
recall = dict()

plt.figure(figsize=(16, 6))    
for i in range(7):
    precision[i], recall[i], _ = precision_recall_curve(y_true_bin[:, i], y_score_bin[:, i])
    plt.plot(recall[i], precision[i], label='Covertype {}'.format(i + 1))

plt.grid()
plt.xlabel('Recall')
plt.ylabel('True Positive Rate / Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='best')

filename = "precision_recall.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()


# Plot AUC-ROC curve for each class
fpr = dict()
tpr = dict()

plt.figure(figsize=(16, 6))    
for i in range(7):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_score_bin[:, i])
    plt.plot(fpr[i], tpr[i], label='Covertype {}'.format(i + 1))

plt.grid()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate / Precision')
plt.title('ROC Curve')
plt.legend(loc='best')
            
filename = "roc_curve.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()


# Partial Dependence

In [ ]:
# potentially iterate over features (and relation ie 0 to 1)

In [ ]:

features, feature_names = [(0,)], [f"Features #{i}" for i in range(X.shape[1])]
deciles = {0: np.linspace(0, 1, num=5)}

pd_results = partial_dependence(
    model, X, features=1, kind="average", grid_resolution=5)

display = PartialDependenceDisplay(
    [pd_results], features=features, feature_names=feature_names,
    target_idx=0, deciles=deciles
)
display.plot(pdp_lim={1: (-1.38, 0.66)})

plt.grid()
plt.xlabel('Feature Value')
plt.ylabel('Partial Dependence') 
plt.title('Partial Dependence')

filename = "partial_dependence.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()


# Confusion Matrix

In [ ]:

class_names = np.unique(y)

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
titles_options = [
    ("Confusion matrix without normalization", None),
    ("Normalized confusion matrix", "true"),
]
for title, normalize in titles_options:
    disp = ConfusionMatrixDisplay.from_estimator(
        model,
        X_test,
        y_test,
        display_labels=class_names + 1,
        cmap=plt.cm.Blues,
        normalize=normalize,
    )
    disp.ax_.set_title(title)

    png_name = title.lower().replace(" ", "_")
    filename = f"{png_name}.png"
    plt.savefig(os.path.join(results_path, filename))

plt.show()

# Shap Values

In [ ]:
explainer = shap.TreeExplainer(model)
explanation = explainer.shap_values(X_test, check_additivity=False)


In [ ]:
shap.summary_plot(explanation, X_test, plot_type="bar", show=False)

filename = f"shap_summary.png"
plt.savefig(os.path.join(results_path, filename))
plt.close()  

SHAP values show how each feature affects each final prediction, the significance of each feature compared to others, and the model's reliance on the interaction between features.


# KAGGLE Prediction

In [ ]:
test_processed = pd.read_csv('Data/test_engineered.csv')

In [ ]:
test_processed.head()

In [ ]:
test_processed = test_processed.loc[:, [col for col in test_processed if not col.startswith('Cover_Type_')]]
X_kaggle = test_processed.drop(columns=['Aspect_Sector'])
y_kaggle = model.predict(X_kaggle) + 1

In [ ]:
pd.read_csv("Data/Kaggle/full_submission.csv").head()

In [ ]:
test_processed['Cover_Type'] = y_kaggle

In [ ]:
kaggle_submission = test_processed.loc[:, ['Id', 'Cover_Type']]

In [ ]:
kaggle_submission.Cover_Type.value_counts()

In [ ]:
kaggle_submission.to_csv(f'Data/kaggle_submission_{timestamp}.csv', index=False)

In [ ]:
pd.read_csv(f'Data/kaggle_submission_{timestamp}.csv')